In [ ]:
import ccdproc
import os
import datetime

from astropy import units as u

from multiprocessing import cpu_count, Lock

def getMkdir(dirPath: str) -> str:
	if not os.path.exists(dirPath):
		os.mkdir(dirPath)
	return dirPath

LOGGING_LOCK = Lock()
def printToLog(msg: str):
	with LOGGING_LOCK:
		with open(os.path.join(BASE_DATA_DIR, f"{OBSV_DATE}.log"), "a+") as logFile:
			logFile.write(f"[{datetime.datetime.now()}] {msg}\n")

MAX_PARALLEL = cpu_count() - 2

OBSV_DATE = datetime.date(year=2022, month=11, day=5).strftime("%Y-%m-%d")
BASE_DATA_DIR = "/run/media/ramon/ROG_Data/Tesis/IturbideDatos/ATOJ339.9469+45.1464/Aviles Observaciones"

BIAS_DIR =  os.path.join(BASE_DATA_DIR, "Bias", OBSV_DATE)
DARKS_DIR = os.path.join(BASE_DATA_DIR, "Darks", OBSV_DATE)
FLATS_DIR = os.path.join(BASE_DATA_DIR, "Flats", OBSV_DATE)

DATA_DIR = [os.path.join(BASE_DATA_DIR, d) for d in os.listdir(BASE_DATA_DIR) if OBSV_DATE in d and os.path.isdir(os.path.join(BASE_DATA_DIR, d))][0]
CCD_KWARGS = {'unit': 'adu'}
DATA_DIR

In [ ]:
# Master Bias

biasImages = ccdproc.ImageFileCollection(BIAS_DIR)
print("Using bias images:", biasImages)
masterBias = ccdproc.Combiner(biasImages.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()

biasOutDir = getMkdir(os.path.join(BIAS_DIR, "out"))
biasFilePath = os.path.join(biasOutDir, f"{OBSV_DATE}_MasterBias.fits")
masterBias.write(biasFilePath, overwrite=True)

In [ ]:
# Master Dark

rawDarks = ccdproc.ImageFileCollection(DARKS_DIR)
print("Using dark images:", rawDarks)
biasCorrectedDir = getMkdir(os.path.join(DARKS_DIR, "bias_corrected"))

exposureTime: None
for dark, dark_fname in rawDarks.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	correctedDark = ccdproc.subtract_bias(dark, masterBias)
	exposureTime = correctedDark.header['EXPOSURE']
	correctedDark.write(os.path.join(biasCorrectedDir, f"b_{dark_fname}"), overwrite=True)

b_darks = ccdproc.ImageFileCollection(biasCorrectedDir)
masterDark = ccdproc.Combiner(b_darks.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()
masterDark.header['EXPOSURE'] = exposureTime

darkOutDir = getMkdir(os.path.join(DARKS_DIR, "out"))
masterDarkFilePath = os.path.join(darkOutDir, f"{OBSV_DATE}_MasterDark.fits")
masterDark.write(masterDarkFilePath, overwrite=True)

In [ ]:
# Master flat

rawFlats = ccdproc.ImageFileCollection(FLATS_DIR)
print("Using flat images:", rawFlats)
correctedFlatsDir = getMkdir(os.path.join(FLATS_DIR, "corrected"))

for flat, flat_fname in rawFlats.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	b_flat = ccdproc.subtract_bias(flat, masterBias)
	bd_flat = ccdproc.subtract_dark(b_flat, masterDark, exposure_time='EXPOSURE', exposure_unit=u.second, scale=True)
	bd_flat.write(os.path.join(correctedFlatsDir, f"bd_{flat_fname}"), overwrite=True)

bd_flats = ccdproc.ImageFileCollection(correctedFlatsDir)
masterFlat = ccdproc.Combiner(bd_flats.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()

flatOutDir = getMkdir(os.path.join(FLATS_DIR, "out"))
masterFlatFilePath = os.path.join(flatOutDir, f"{OBSV_DATE}_MasterFlat.fits")
masterFlat.write(masterFlatFilePath, overwrite=True)

In [ ]:
# Bias, Dark, Flat image correction

correctedDir = getMkdir(os.path.join(DATA_DIR, "corrected"))

rawImages = ccdproc.ImageFileCollection(DATA_DIR)
for img, img_fname in rawImages.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	b_img = ccdproc.subtract_bias(img, masterBias)
	bd_img = ccdproc.subtract_dark(b_img, masterDark, exposure_time='EXPOSURE', exposure_unit=u.second, scale=True)
	bdf_img = ccdproc.flat_correct(bd_img, masterFlat)
	bdf_img.write(os.path.join(correctedDir, f"bdf_{img_fname}"), overwrite=True)

In [ ]:
# Plate solving

import subprocess
import datetime
import numpy as np

from multiprocessing import Pool

SOLVE_CMD_TEMPLATE = "solve-field --fits-image --no-plots --timestamp --new-fits {solvedOutPath} --cpulimit 180 -D {tempFiles} {inputPath} --overwrite"
		
def plateSolveFile(correctedFile: str, correctedDir: str):
	solvedFitsDir = getMkdir(os.path.join(correctedDir, "solved-fits"))
	plateSolveCmd = SOLVE_CMD_TEMPLATE.format(inputPath=os.path.join(correctedDir, correctedFile), 
											solvedOutPath=os.path.join(solvedFitsDir, correctedFile),
											tempFiles=os.path.join(correctedDir, "solved"))
	print(plateSolveCmd)
	result = subprocess.run(plateSolveCmd, shell=True)
	if result.returncode != 0:
		printToLog(f"Failed to plate solve {correctedFile}")

def plateSolve(correctedFiles: list[str], correctedDir: str):
	# print(f"{os.getpid()} {correctedFiles[0]} {len(correctedFiles)}\n")
	printToLog(f"[{os.getpid()}] Begin plate solving")
	for cFile in correctedFiles:
		plateSolveFile(cFile, correctedDir)
	printToLog(f"[{os.getpid()}] Finished plate solving")

class SolveCaller:
	def __init__(self, correctedDir: str) -> None:
		self.correctedDir = correctedDir

	def __call__(self, imagesChunks):
		plateSolve(imagesChunks, self.correctedDir)

correctedDir = os.path.join(DATA_DIR, "corrected") # output files from previous cell
imagesChunks = np.array_split(os.listdir(correctedDir), MAX_PARALLEL)
with Pool(MAX_PARALLEL) as pool:
	pool.map(SolveCaller(correctedDir), imagesChunks)

In [ ]:
# Shift plate solved images

from ccdproc import wcs_project
from astropy.wcs import WCS

from astropy.io import fits

targetWcs: WCS = None
correctedDir = os.path.join(DATA_DIR, "corrected")
solvedFitsDir = os.path.join(correctedDir, "solved-fits")
shiftedDir = getMkdir(os.path.join(correctedDir, "shifted"))

rawImages = ccdproc.ImageFileCollection(solvedFitsDir)
for img, img_fname in rawImages.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	if not targetWcs:
		targetWcs = WCS(img.header)
		with fits.open(os.path.join(solvedFitsDir, img_fname)) as fitsImg:
			targetWcs = WCS(fitsImg.pop())
		img.write(os.path.join(shiftedDir, f"s_{img_fname}"), overwrite=True)
	else:
		shiftedImg = wcs_project(img, targetWcs)
		shiftedImg.write(os.path.join(shiftedDir, f"s_{img_fname}"), overwrite=True)

In [ ]:
# # Period analysis

# from astropy.timeseries import TimeSeries
# from astropy.io import fits

# # Testing, should be gotten from previous cell
# correctedDir = os.path.join(DATA_DIR, "corrected")
# shiftedDir = os.path.join(correctedDir, "shifted")

# testFitsPath = os.path.join(shiftedDir, os.listdir(shiftedDir)[0])
# with fits.open(testFitsPath) as fitsFile:
# 	# print(fitsFile.info(), "--------------------", sep='\n')
# 	priheader = fitsFile[0].header
# 	print(repr(priheader))
# 	exposureTime = priheader['EXPOSURE'] * u.second
# 	photflam = fitsFile
# 	sciData = fitsFile[0].data
# 	maskData = fitsFile[1].data
# 	# print(priheader, "--------------------", sep='\n')
# 	# print(sciData)
# 	# print(maskData)